In [92]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import os


In [93]:
#ImageDataGenerator implements functions useful for input image scaling and augmentation -- you may want more!

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


In [94]:
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=True,
        seed=1953)


valid_generator = valid_datagen.flow_from_directory(
        'data/validation',
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=False, #don't shuffle or label results will be all wrong
        seed=1953)


test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(32, 32),
    color_mode='rgb',
    batch_size=1,
    class_mode='categorical',
    shuffle=False, #don't shuffle or label results will be all wrong
    seed=1953)


Found 2744 images belonging to 8 classes.
Found 929 images belonging to 8 classes.
Found 884 images belonging to 8 classes.


In [95]:
# neural network model
# you may want to vary these parameters, etc

num_classes = 8 # fixed by the number of classes of signs that we gave you. Dont change

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [96]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam', # may want to try others
              metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_65 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_66 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 4096)              0         
__________

In [97]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4 # may need to increase if not seeing low enough losses
)


Epoch 1/4
2744/2744 [==============================] - 13s 5ms/step - loss: 0.4459 - acc: 0.8633 - val_loss: 0.1615 - val_acc: 0.9559
Epoch 2/4
2744/2744 [==============================] - 12s 4ms/step - loss: 0.1121 - acc: 0.9647 - val_loss: 0.2127 - val_acc: 0.9333
Epoch 3/4
2744/2744 [==============================] - 12s 4ms/step - loss: 0.0861 - acc: 0.9745 - val_loss: 0.3195 - val_acc: 0.9053
Epoch 4/4
2744/2744 [==============================] - 12s 5ms/step - loss: 0.0892 - acc: 0.9752 - val_loss: 0.1710 - val_acc: 0.9559


In [98]:
Y_pred = model.predict_generator(valid_generator, steps=STEP_SIZE_VALID)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Classification Report')
print(classification_report(valid_generator.classes, y_pred, target_names=valid_generator.class_indices))

Confusion Matrix
[[ 40   7   1   0   0   0   1   3]
 [  0  38  12   0   0   0   0   0]
 [  0   0 186  11   0   0   3   0]
 [  0   1   5 131   9   1   7   0]
 [  0   0   0   0   6  11   0   0]
 [  0   0   0   0   0  36  11   0]
 [  0   0   3   5   0   0 374   5]
 [ 11   0   0   1   0   0   3   7]]
Classification Report
              precision    recall  f1-score   support

   keepRight       0.78      0.77      0.78        52
       merge       0.83      0.76      0.79        50
  pedestrian       0.90      0.93      0.91       200
 signalAhead       0.89      0.85      0.87       154
speedLimit25       0.40      0.35      0.38        17
speedLimit35       0.75      0.77      0.76        47
        stop       0.94      0.97      0.95       387
       yield       0.47      0.32      0.38        22

   micro avg       0.88      0.88      0.88       929
   macro avg       0.74      0.71      0.73       929
weighted avg       0.88      0.88      0.88       929



In [99]:
#USE CELLS BELOW TO COMPLETE THE EXERCISE WITH THE TEST SET

In [100]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
Y_pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=train_generator.class_indices))

Confusion Matrix
[[ 58   0   0   0   0   0   0   0]
 [  0  48   4   1   0   0   0   2]
 [  0   0 208   0   0   0   0   0]
 [  0   0   3 134   0   0   8   1]
 [  0   0   0   0  21   3   1   0]
 [  0   0   0   0   0  41   0   0]
 [  0   0   0   2   0   0 329   0]
 [  0   0   0   2   0   0   9   9]]
Classification Report
              precision    recall  f1-score   support

   keepRight       1.00      1.00      1.00        58
       merge       1.00      0.87      0.93        55
  pedestrian       0.97      1.00      0.98       208
 signalAhead       0.96      0.92      0.94       146
speedLimit25       1.00      0.84      0.91        25
speedLimit35       0.93      1.00      0.96        41
        stop       0.95      0.99      0.97       331
       yield       0.75      0.45      0.56        20

   micro avg       0.96      0.96      0.96       884
   macro avg       0.95      0.88      0.91       884
weighted avg       0.96      0.96      0.96       884



In [ ]:
#This cell dumps out a file of which files were incorrectly predicted
#so you can see if you need more features, more training samples, etc
import pandas as pd
predicted_class_indices=np.argmax(Y_pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)